The goal is to take every row from the conditional data and add macula ids for each row for both the protasis and apodosis separately.

Ingest excel sheets

In [1]:
import pandas as pd
import openai
import getpass
import time
import json
from lxml import etree
import requests

In [2]:
# Set the maximum number of rows and columns to display (set them to None for unlimited)
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
pd.set_option('display.max_colwidth', None)

In [4]:
ids_file = 'macula_ids_with_glosses.tsv'
excel_path = 'FINAL_all_conditionals_with_verse_data_and_alignment.tsv'

In [5]:
cond_df = pd.read_csv(excel_path, sep='\t')
cond_df.head()

,reference,class,inv.,probability,time orientation,illocutionary force,english translations,notes,parallel passage(s),unnamed: 10,unnamed:,parallel passages,unnamed: 9,b_ch_v,scope,protasis,apodosis,matched_protasis_words,matched_apodosis_words,all_verse_word_tuples
0,MAT 4:3,1,NaN,Factual,Present,Exhort,"ESV, NASB, NRSV, NIV, NLT: ""if""","P presents a fact that the temptor knew to be true. The devil certainly knows Jesus' identity. Hagner (1993) notes that this might as well be translated ""since"", as the devil is testing Jesus' obedience to the Father rather than questioning his identity. Q presents the subsequent command Jesus is being tempted to obey. The conditional as a whole amounts to a taunt or a challenge. Fong (2014: 30-31) discusses this as an example given by Young (1994) of a rhetorical conditional. That is, it is not being presented as a real condition, but the conditional is used for its logical connection in order to affect some other speech act, in this case ""manipulation"", according to Young. Fong, however, thinks it would be better classified as a ""challenge.""",Luke 4:3,NaN,NaN,NaN,NaN,MAT 4:3,p: (if you are the Son of God)\nq: (command these stones to become bread),[],[],"[['n40004003007', 'If', 'Εἰ'], ['n40004003008', 'Son', 'υἱὸς'], ['n40004003009', 'You are', 'εἶ'], ['n40004003010', '-', 'τοῦ'], ['n40004003011', 'of God', 'θεοῦ']]","[['n40004003012', 'speak', 'εἰπὲ'], ['n40004003013', 'that', 'ἵνα'], ['n40004003014', 'the', 'οἱ'], ['n40004003015', 'stones', 'λίθοι'], ['n40004003016', 'these', 'οὗτοι'], ['n40004003017', 'loaves of bread', 'ἄρτοι'], ['n40004003018', 'might become', 'γένωνται']]","[('n40004003001', 'And', 'καὶ'), ('n40004003002', 'having come', 'προσελθὼν'), ('n40004003003', 'the', 'ὁ'), ('n40004003004', '[one] tempting', 'πειράζων'), ('n40004003005', 'he said', 'εἶπεν'), ('n40004003006', 'to Him', 'αὐτῷ'), ('n40004003007', 'If', 'Εἰ'), ('n40004003008', 'Son', 'υἱὸς'), ('n40004003009', 'You are', 'εἶ'), ('n40004003010', '-', 'τοῦ'), ('n40004003011', 'of God', 'θεοῦ'), ('n40004003012', 'speak', 'εἰπὲ'), ('n40004003013', 'that', 'ἵνα'), ('n40004003014', 'the', 'οἱ'), ('n40004003015', 'stones', 'λίθοι'), ('n40004003016', 'these', 'οὗτοι'), ('n40004003017', 'loaves of bread', 'ἄρτοι'), ('n40004003018', 'might become', 'γένωνται')]"
1,MAT 4:6,1,NaN,Factual,Present,Exhort,"ESV, NASB, NRSV, NIV, NLT: ""if""","As with 4:3, this conditional expresses no doubt about Jesus' identity, but is being used to manipulate, tempt, and challenge him.",Luke 4:9,NaN,NaN,NaN,NaN,MAT 4:6,p: (if you are the Son of God)\nq: (throw yourself down),[],[],"[['n40004006004', 'If', 'Εἰ'], ['n40004006005', 'Son', 'υἱὸς'], ['n40004006006', 'You are', 'εἶ'], ['n40004006007', '-', 'τοῦ'], ['n40004006008', 'of God', 'θεοῦ']]","[['n40004006009', 'throw', 'βάλε'], ['n40004006010', 'Yourself', 'σεαυτὸν'], ['n40004006011', 'down', 'κάτω']]","[('n40004006001', 'and', 'καὶ'), ('n40004006002', 'says', 'λέγει'), ('n40004006003', 'to him', 'αὐτῷ'), ('n40004006004', 'If', 'Εἰ'), ('n40004006005', 'Son', 'υἱὸς'), ('n40004006006', 'You are', 'εἶ'), ('n40004006007', '-', 'τοῦ'), ('n40004006008', 'of God', 'θεοῦ'), ('n40004006009', 'throw', 'βάλε'), ('n40004006010', 'Yourself', 'σεαυτὸν'), ('n40004006011', 'down', 'κάτω'), ('n40004006012', 'it has been written', 'γέγραπται'), ('n40004006013', 'for', 'γὰρ'), ('n40004006014', '-', 'ὅτι'), ('n40004006015', 'To the', 'Τοῖς'), ('n40004006016', 'angels', 'ἀγγέλοις'), ('n40004006017', 'of Him', 'αὐτοῦ'), ('n40004006018', 'He will give orders', 'ἐντελεῖται'), ('n40004006019', 'concerning', 'περὶ'), ('n40004006020', 'You', 'σοῦ'), ('n40004006021', 'and', 'καὶ'), ('n40004006022', 'in', 'ἐπὶ'), ('n40004006023', '[their] hands', 'χειρῶν'), ('n40004006024', 'will they bear up', 'ἀροῦσίν'), ('n40004006025', 'You', 'σε'), ('n40004006026', '', 'μήποτε'), ('n40004006027', 'You strike', 'προσκόψῃς'), ('n40004006028', 'against', 'πρὸς'), ('n40004006029', 'a stone', 'λίθον'), ('n40004006030', 'the', 'τὸν

In [13]:
inconsistent_ids_map = {
    "MAT 6:23":"MAT 6:23",
"MAT 10:13b":"MAT 10:13",
"MAT 18:15b":"MAT 18:15",
"MAT 18:17b":"MAT 18:17",
"MAT 21:21b":"MAT 21:21",
"MAT 24:26b":"MAT 24:26",
"MAT 26:42b":"MAT 26:42",
"MAT 5:13b":"MAT 5:13",
"MAT 11:27b":"MAT 11:27",
"MAT 11:27c":"MAT 11:27",
"MAT 16:19b":"MAT 16:19",
"MAT 18:18b":"MAT 18:18",
"MAT 18:19b":"MAT 18:19",
"MRK 7:11b":"MRK 7:11",
"MRK 8:35":"MRK 8:35",
"MRK 11:13b":"MRK 11:13",
"LUK 10:6b":"LUK 10:6",
"LUK 13:9b":"LUK 13:9",
"LUK 16:31b":"LUK 16:31",
"LUK 17:3b":"LUK 17:3",
"LUK 22:67b":"LUK 22:67",
"LUK 4:6":"LUK 4:6",
"LUK 9:48":"LUK 9:48",
"LUK 10:22b":"LUK 10:22",
"LUK 10:22c":"LUK 10:22",
"LUK 23:06":"LUK 23:06",
"JHN 3:12b":"JHN 3:12",
"JHN 10:38b":"JHN 10:38",
"JHN 12:24b":"JHN 12:24",
"JHN 12:26b":"JHN 12:26",
"JHN 13:17b":"JHN 13:17",
"JHN 15:4b":"JHN 15:4",
"JHN 15:20c":"JHN 15:20",
"JHN 16:7b":"JHN 16:7",
"JHN 18:23b":"JHN 18:23",
"JHN 20:23":"JHN 20:23",
"JHN 15:7b":"JHN 15:7",
"ACT 25:11b":"ACT 25:11",
"ROM 2:25b":"ROM 2:25",
"ROM 7:3b":"ROM 7:3",
"ROM 7:7b":"ROM 7:7",
"ROM 8:9b":"ROM 8:9",
"ROM 8:13b":"ROM 8:13",
"ROM 8:17b":"ROM 8:17",
"ROM 11:16b":"ROM 11:16",
"ROM 12:7b":"ROM 12:7",
"ROM 12:20b":"ROM 12:20",
"ROM 14:8b":"ROM 14:8",
"ROM 11:6b":"ROM 11:6",
"ROM 11:22b":"ROM 11:22",
"1CO 7:28b":"1CO 7:28",
"1CO 8:8b":"1CO 8:8",
"1CO 9:11b":"1CO 9:11",
"1CO 9:16b":"1CO 9:16",
"1CO 9:17b":"1CO 9:17",
"1CO 11:6b":"1CO 11:6",
"1CO 12:17b":"1CO 12:17",
"1CO 12:26b":"1CO 12:26",
"1CO 14:6b":"1CO 14:6",
"1CO 15:2b":"1CO 15:2",
"1CO 15:32b":"1CO 15:32",
"1CO 16:10":"1CO 16:10",
"1CO 2:11b":"1CO 2:11",
"1CO 8:5":"1CO 8:5",
"1CO 13:8a":"1CO 13:8",
"1CO 14:7a":"1CO 14:7",
"2CO 1:6b":"2CO 1:6",
"2CO 5:13b":"2CO 5:13",
"2CO 7:8b":"2CO 7:8",
"2CO 7:8c":"2CO 7:8",
"2CO 12:15b":"2CO 12:15",
"2CO 2:2b":"2CO 2:2",
"2CO 8:12b":"2CO 8:12",
"2CO 13:5a":"2CO 13:5",
"GAL 6:9":"GAL 6:9",
"12TH 1Th 3:8":"1TH 3:8",
"12TH 1Th 4:14":"1TH 4:14",
"12TH 2Th 2:3":"2TH 2:3",
"12TH 2Th 3:10b":"2TH 3:10",
"12TH 2Th 3:14":"2TH 3:14",
"12TH 1Th 2:7":"1TH 2:7",
"12TH 1Th 5:10":"1TH 5:10",
"12TH 2Th 1:5-7":"2TH 1:5-7",
"12TH 2Th 2:15":"2TH 2:15",
"12TI-TIT-PHM 1 Tim 1:8":"1TI 1:8",
"12TI-TIT-PHM 1 Tim 2:15":"1TI 2:15",
"12TI-TIT-PHM 1 Tim 3:1":"1TI 3:1",
"12TI-TIT-PHM 1 Tim 3:5":"1TI 3:5",
"12TI-TIT-PHM 1 Tim 3:15":"1TI 3:15",
"12TI-TIT-PHM 1 Tim 5:4":"1TI 5:4",
"12TI-TIT-PHM 1 Tim 5:8":"1TI 5:8",
"12TI-TIT-PHM 1 Tim 5:9-10":"1TI 5:9-10",
"12TI-TIT-PHM 1 Timothy 5:16":"1TI 5:16",
"12TI-TIT-PHM 1 Timothy 5:19":"1TI 5:19",
"12TI-TIT-PHM 1 Tim 6:3":"1TI 6:3",
"12TI-TIT-PHM 1 Tim 1:10":"1TI 1:10",
"12TI-TIT-PHM 2 Tim 2:5":"2TI 2:5",
"12TI-TIT-PHM 2 Tim 2:11":"2TI 2:11",
"12TI-TIT-PHM 2 Tim 2:12a":"2TI 2:12",
"12TI-TIT-PHM 2 Tim 2:12b":"2TI 2:12",
"12TI-TIT-PHM 2 Tim 2:13":"2TI 2:13",
"12TI-TIT-PHM 2 Tim 2:21":"2TI 2:21",
"12TI-TIT-PHM Titus 1:6":"Titus 1:6",
"12TI-TIT-PHM Philemon 17":"PHM 17",
"12TI-TIT-PHM Philemon 18":"PHM 18",
"JAS 0.0576388888888889":"JAS 1:23",
"JAS 0.0597222222222222":"JAS 1:26",
"12PE 1 Peter 1:6":"1PE 1:6",
"12PE 1 Peter 1:17":"1PE 1:17",
"12PE 1 Peter 2:2-3":"1PE 2:2-3",
"12PE 1 Peter 2:19":"1PE 2:19",
"12PE 1 Peter 2:20a":"1PE 2:20",
"12PE 1 Peter 2:20b":"1PE 2:20",
"12PE 1 Peter 3:1":"1PE 3:1",
"12PE 1 Peter 3:13":"1PE 3:13",
"12PE 1 Peter 3:14":"1PE 3:14",
"12PE 1 Peter 3:17":"1PE 3:17",
"12PE 1 Peter 4:11a":"1PE 4:11",
"12PE 1 Peter 4:11b":"1PE 4:11",
"12PE 1 Peter 4:14":"1PE 4:14",
"12PE 1 Peter 4:16":"1PE 4:16",
"12PE 1 Peter 4:17b":"1PE 4:17",
"12PE 1 Peter 4:18":"1PE 4:18",
"12PE 2 Peter 2:4-9":"2PE 2:4-9",
"12PE 2 Peter 2:20":"2PE 2:20",
"12PE 1 Peter 2:13-14":"1PE 2:13-14",
"123JN 1 John 1:6":"1JN 1:6",
"123JN 1 John 1:7":"1JN 1:7",
"123JN 1 John 1:8":"1JN 1:8",
"123JN 1 John 1:9":"1JN 1:9",
"123JN 1 John 1:10":"1JN 1:10",
"123JN 1 John 2:1b":"1JN 2:1",
"123JN 1 John 2:3":"1JN 2:3",
"123JN 1 John 2:15b":"1JN 2:15",
"123JN 1 John 2:19b":"1JN 2:19",
"123JN 1 John 2:24b":"1JN 2:24",
"123JN 1 John 2:29":"1JN 2:29",
"123JN 1 John 3:13":"1JN 3:13",
"123JN 1 John 3:20":"1JN 3:20",
"123JN 1 John 3:21":"1JN 3:21",
"123JN 1 John 4:11":"1JN 4:11",
"123JN 1 John 4:12":"1JN 4:12",
"123JN 1 John 4:20":"1JN 4:20",
"123JN 1 John 5:9":"1JN 5:9",
"123JN 1 John 5:14b":"1JN 5:14",
"123JN 1 John 5:15a":"1JN 5:15",
"123JN 1 John 5:16":"1JN 5:16",
"123JN 2 John 10":"2JN 10",
"123JN 3 John 10":"3JN 10",
"123JN 1 John 2:22":"1JN 2:22",
"123JN 1 John 2:28":"1JN 2:28",
"123JN 1 John 3:2":"1JN 3:2",
"123JN 1 John 3:22":"1JN 3:22",
"123JN 1 John 4:1":"1JN 4:1",
"123JN 1 John 4:15":"1JN 4:15",
"123JN 1 John 5:5":"1JN 5:5",
"123JN 1 John 5:15b":"1JN 5:15",
"123JN 3 John 5":"3JN 5",
"REV 2:5c":"REV 2:5",
"REV 11:5b":"REV 11:5",
"REV 13:10b":"REV 13:10",
"REV 9:4":"REV 9:4",
}

In [43]:
# Concatenate all DataFrames in df_list
# concatenated_df = pd.concat(df_list, ignore_index=True)

# Your existing code, adjusted for the concatenated DataFrame
mark_df = cond_df

ids_df = pd.read_csv(ids_file, sep='\t')

old_refs = mark_df['reference'].tolist()
new_refs = []

for i in old_refs:
    if i in inconsistent_ids_map:
        new_refs.append(inconsistent_ids_map[i])
    else:
        new_refs.append(i)

mark_df['reference'] = new_refs

ids_df['b_ch_v'] = ids_df['ref'].str.extract(r'(\w+ \d+:\d+)')
mark_df['b_ch_v'] = mark_df['reference'].str.extract(r'(\w+ \d+:\d+)')

# create a subset of the ifs_df for rows where the 'b_ch_v' value is a substring within the 'Reference' column of the mark_df
# e.g., 1:1 is a substring of MRK 1:1!1
subset = ids_df[ids_df['b_ch_v'].isin(mark_df['reference'])] 

# fill all NaN in both dataframes with empty strings
mark_df = mark_df.fillna('')

In [44]:
backup_df = mark_df.copy()

In [8]:
# Dump mark_df to csv
mark_df.to_csv('mark_df.tsv', sep='\t', index=False)

Now we need to try to match up strings. 
Here's an example. 

in the `Scope of conditional (ESV unless noted)` column, we have the string:

"p: (If you will) q: (you can make me clean)"
`p: \(.*?\)` matches the protasis
`q: \(.*?\)` matches the apodosis

Now, in the ids_df, for rows where ids_df[ids_df['b_ch_v'].isin(mark_df['Reference'])] (i.e., the subset relevant to just one row in mark_df) is true, we have the rows:

xml:id	ref	english	gloss	text	b_ch_v
603	n41001040001	MRK 1:40!1	and	And	Καὶ	1:40
604	n41001040002	MRK 1:40!2	came	comes	ἔρχεται	1:40
605	n41001040003	MRK 1:40!3	to	to	πρὸς	1:40
606	n41001040004	MRK 1:40!4	him	Him	αὐτὸν	1:40
607	n41001040005	MRK 1:40!5	leper	a leper	λεπρὸς	1:40

So, here we note that there is both an 'english' and 'gloss' column.

We want to find all ifs_df rows where the 'b_ch_v' value is a substring of the 'Reference' column in the mark_df, and then we want to take the 'english' and 'gloss' values from the ids_df and match them up with any English words (strip brackets and square brackets on both sides of the equation) into the 'Scope of conditional (ESV unless noted)' column in the mark_df.
Then, we want to populate the columns 'macula_ids_in_protasis' and 'macula_ids_in_apodosis' with the 'xml:id' values from the ids_df.

In [46]:
verse_df = ids_df[ids_df['b_ch_v'] == mark_df.iloc[3]['b_ch_v']]
verse_df

,xml:id,ref,english,gloss,text,b_ch_v
1783,n40005013001,MAT 5:13!1,you,You,Ὑμεῖς,MAT 5:13
1784,n40005013002,MAT 5:13!2,are,are,ἐστε,MAT 5:13
1785,n40005013003,MAT 5:13!3,the,the,τὸ,MAT 5:13
1786,n40005013004,MAT 5:13!4,salt,salt,ἅλας,MAT 5:13
1787,n40005013005,MAT 5:13!5,the,of the,τῆς,MAT 5:13
1788,n40005013006,MAT 5:13!6,earth,earth,γῆς,MAT 5:13
1789,n40005013007,MAT 5:13!7,if,if,ἐὰν,MAT 5:13
1790,n40005013008,MAT 5:13!8,but,however,δὲ,MAT 5:13
1791,n40005013009,MAT 5:13!9,the,the,τὸ,MAT 5:13
1792,n40005013010,MAT 5:13!10,salt,salt,ἅλας,MAT 5:13


In [11]:
import re
# For every b_ch_v in mark_df, find the corresponding rows in ids_df
# p_q = mark_df.loc[mark_df['b_ch_v'] == verse]
# let's make columns for p and q content respectively
mark_df['protasis'] = [[] for _ in range(len(mark_df))]
mark_df['apodosis'] = [[] for _ in range(len(mark_df))]
mark_df['all_verse_word_tuples'] = [[] for _ in range(len(mark_df))]

for i, row in enumerate(mark_df):
    # populate the protasis and apodosis columns with the corresponding values from the scope column
    # extract using regex from, e.g., q:(All these I will give you)\np: (if you will fall down and worship me)
    protasis_match = re.search(r'p: ?(.*)', mark_df['scope'][i])
    apodosis_match = re.search(r'q: ?(.*)', mark_df['scope'][i])
    protasis_string = protasis_match.group(1) if protasis_match else None
    apodosis_string = apodosis_match.group(1) if apodosis_match else None
    
    if protasis_string:
        mark_df['protasis'][row] = protasis_string
    else:
        mark_df['protasis'][row] = ''
    
    if apodosis_string:
        mark_df['apodosis'][row] = apodosis_string
    else:
        mark_df['apodosis'][row] = ''
    
    verse_df = ids_df[ids_df['reference'] == mark_df.iloc[i]['reference']]
    verse_df = verse_df.fillna('')
    
    # let's make a list of tuples of all the words in the verse
    verse_word_tuples = []
    for index, row in verse_df.iterrows():
        verse_word_tuples.append((row['xml:id'], row['gloss'], row['text']))
    
    mark_df.at[i, 'all_verse_word_tuples'] = verse_word_tuples
    
    # prompt = f''' {protasis_string}\n{apodosis_string}\n\nProtasis:'''
    
mark_df.to_csv('all_conditionals_with_verse_data.tsv', sep='\t', index=False)

In [47]:
mark_df[mark_df['b_ch_v'] == 'MAT 8:2']

,reference,class,inv.,probability,time orientation,illocutionary force,english translations,notes,parallel passage(s),unnamed: 10,unnamed:,parallel passages,unnamed: 9,b_ch_v,scope,protasis,apodosis,matched_protasis_words,matched_apodosis_words,all_verse_word_tuples
20,MAT 8:2,3,,Neutral,Present,Request,"ESV, NASB, NRSV, NIV, NLT: ""if""","Jesus is able to make the man clean regardless of his will to do so. Many commentators agree that the man is not expressing any doubt as to Jesus' power, but the conditional merely expresses his doubt as to Jesus' willingness.",Mark 1:40; Luke 5:12,,,,,MAT 8:2,p: (if you will)\nq: (you can make me clean),[],[],"[['n40008002009', 'if', 'ἐὰν'], ['n40008002010', 'You are willing', 'θέλῃς']]","[['n40008002011', 'You are able', 'δύνασαί'], ['n40008002012', 'me', 'με'], ['n40008002013', 'to cleanse', 'καθαρίσαι']]",[]


In [14]:
def generate_prompt(verse):
    should_print = False
    if verse in inconsistent_ids_map.keys():
        print('Rewriting verse', verse, 'to', inconsistent_ids_map[verse])
        verse = inconsistent_ids_map[verse]
        # should_print = True
    p_q = mark_df.loc[mark_df['b_ch_v'] == verse]
    verse_df = ids_df[ids_df['b_ch_v'] == verse]    
    verse_word_tuples = []
    for index, row in verse_df.iterrows():
        verse_word_tuples.append((row['gloss'], row['text'], row['xml:id']))
    
    scope = p_q['scope']
    if scope.values.size < 1:
        print(p_q)
        raise ValueError('scope.values is empty')
    
    p_q_content = scope.values[0] if scope.values[0] else None
    if not p_q_content or len(verse_word_tuples) == 0:
        raise ValueError('Either p_q_content or verse_word_tuples is empty.', p_q_content, verse_word_tuples)                        
    
    formatted_word_tuples = '\n'.join([str(tuple) for tuple in verse_word_tuples])

    prompt = f'''## Instruction:
Use the rows from the table below to associate xml:ids with the protasis and apodosis listed below. Note that the English words won't always match up exactly (since they come from different English translations of the underlying Greek), so try to find the closest matches to the entire phrase in the protasis or apodosis.

Here is an example:

## Context

xml:id	english	gloss	text	b_ch_v
n40004003001	and	And	καὶ	MAT 4:3
n40004003002	came	having come	προσελθὼν	MAT 4:3
n40004003003	the	the	ὁ	MAT 4:3
n40004003004	tempter	[one] tempting	πειράζων	MAT 4:3
n40004003005	said	he said	εἶπεν	MAT 4:3
n40004003006	him	to Him	αὐτῷ	MAT 4:3
n40004003007	if	If	Εἰ	MAT 4:3
n40004003008	son	Son	υἱὸς	MAT 4:3
n40004003009	are	You are	εἶ	MAT 4:3
n40004003010	NaN	-	τοῦ	MAT 4:3
n40004003011	God	of God	θεοῦ	MAT 4:3
n40004003012	command	speak	εἰπὲ	MAT 4:3
n40004003013	to	that	ἵνα	MAT 4:3
n40004003014	NaN	the	οἱ	MAT 4:3
n40004003015	stones	stones	λίθοι	MAT 4:3
n40004003016	these	these	οὗτοι	MAT 4:3
n40004003017	bread	loaves of bread	ἄρτοι	MAT 4:3
n40004003018	become	might become	γένωνται	MAT 4:3

p: if you are the Son of God
q: command these stones to become bread

## Results (best match; some words left out if they are not part of the protasis or apodosis; some words only semantically similar but a good match nevertheless):
```
{{
    "protasis": [
        ('n40004003007', 'If', 'Εἰ'),
        ('n40004003008', 'Son', 'υἱὸς'), 
        ('n40004003009', 'You are', 'εἶ'), 
        ('n40004003010', '-', 'τοῦ'),
        ('n40004003011', 'of God', 'θεοῦ'),
    ]
    "apodosis": [
        ('n40004003012', 'speak', 'εἰπὲ'), 
        ('n40004003013', 'that', 'ἵνα'), 
        ('n40004003014', 'the', 'οἱ'), 
        ('n40004003015', 'stones', 'λίθοι'), 
        ('n40004003016', 'these', 'οὗτοι'), 
        ('n40004003017', 'loaves of bread', 'ἄρτοι'), 
        ('n40004003018', 'might become', 'γένωνται'),
    ]
}}
```

Now follow a similar format with this protasis and apodosis pair using perfect JSON:

## Context 
{verse_df}

{p_q_content}

## Results:
'''
    if should_print:
        print(prompt)
    return prompt

In [16]:
openai.api_key = getpass.getpass(prompt='Please enter your OpenAI API key.')
openai.organization = getpass.getpass(prompt='Please enter your OpenAI Organization key.')

In [17]:
# define your GPT completion function
model = 'gpt-3.5-turbo' # 'gpt-4-1106-preview' #3.5-turbo

import ast

MAX_RETRIES = 10
def align(prompt):
    system_prompt = "Analyze the p-q phrases and align the individual words to ids with the table the user provides. Note that some verses may have more than one p-q pair, so just use the words that match the complete wording. The other pairs will be handled in subsequent prompts."
    messages = [
        {"role": 'system', "content": system_prompt},
        {"role": 'user', 'content': prompt}
    ]
    for i in range(MAX_RETRIES):
        try:
            response = openai.ChatCompletion.create(
                model=model,
                messages=messages,
                temperature=0,
            )
            generated_texts = [
                choice.message["content"] for choice in response["choices"]
            ]
            result = ast.literal_eval(generated_texts[0])
            return result
        except Exception as e:
            print('Error in alignment:', e)
            if i < MAX_RETRIES - 1:  # i is zero indexed
                # time.sleep(1)
                continue
            else:
                return {"error": str(e)}
        

In [48]:
refs = mark_df['b_ch_v'].tolist()
refs[-100:]

['JAS 3:3',
 'JAS 3:14',
 'JAS 4:11',
 'JAS 4:15',
 'JAS 5:15',
 'JAS 5:19',
 '',
 '',
 '',
 'JAS 4:4',
 '1PE 1:6',
 '1PE 1:17',
 '1PE 2:2',
 '1PE 2:19',
 '1PE 2:20',
 '1PE 2:20',
 '1PE 3:1',
 '1PE 3:13',
 '1PE 3:14',
 '1PE 3:17',
 '1PE 4:11',
 '1PE 4:11',
 '1PE 4:14',
 '1PE 4:16',
 '1PE 4:17',
 '1PE 4:18',
 '',
 '',
 '2PE 2:4',
 '2PE 2:20',
 '',
 '',
 '',
 '1PE 2:13',
 '1JN 1:6',
 '1JN 1:7',
 '1JN 1:8',
 '1JN 1:9',
 '1JN 1:10',
 '1JN 2:1',
 '1JN 2:3',
 '1JN 2:15',
 '1JN 2:19',
 '1JN 2:24',
 '1JN 2:29',
 '1JN 3:13',
 '1JN 3:20',
 '1JN 3:21',
 '1JN 4:11',
 '1JN 4:12',
 '1JN 4:20',
 '1JN 5:9',
 '1JN 5:14',
 '1JN 5:15',
 '1JN 5:16',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '1JN 2:22',
 '1JN 2:28',
 '1JN 3:2',
 '1JN 3:22',
 '1JN 4:1',
 '1JN 4:15',
 '1JN 5:5',
 '1JN 5:15',
 '',
 'REV 2:5',
 'REV 2:5',
 'REV 2:16',
 'REV 2:22',
 'REV 3:3',
 'REV 3:20',
 'REV 11:5',
 'REV 11:5',
 'REV 13:9',
 'REV 13:10',
 'REV 13:10',
 'REV 13:17',
 'REV 14:9',
 'REV 20:15',
 'REV 22:18',
 'REV 22:19',


In [19]:
refs[20]

'MAT 8:2'

In [49]:
len(inconsistent_ids_map)

162

In [21]:
result_list = []

In [50]:
existing_refs = set()
with open('output_Nov_14_nathan.jsonl', 'r', encoding='utf-8') as file:
    for line in file:
        data = json.loads(line)
        existing_refs.add(data['vref'])

In [52]:
for i, ref in enumerate(refs):
    if ref not in existing_refs:
        
        print('Processing', ref, i, 'of', len(refs))
        
        try:
            prompt = generate_prompt(ref)
        except ValueError as e:
            print('Error in prompt generation:', e)
            continue
        
        if not prompt:
            print('Prompt is empty for', ref)
            pass
        results = align(prompt)
        
        try:
            # extract the json from between the triple backticks
            if not type(results) == dict:
                raise ValueError('results is not a dict:', results)
            else:
                # store results
                result_list.append((ref, results))
                print(ref, results)
                with open('output_Nov_14_nathan.jsonl', 'a', encoding='utf-8') as file:
                    json_output = json.dumps({"vref": ref, "data": results}, ensure_ascii=False)
                    file.write(json_output)
                    file.write('\n')
                # with open('pq_macula.jsonl', 'a', encoding='utf-8') as file:
                #     print('made it', results)
                    # json_output = json.dumps(results, ensure_ascii=False)
                    # file.write(results)
                    # file.write('\n')
        except Exception as e:
            print('Error in result parsing:', e)

Processing  82 of 945
Error in prompt generation: ('Either p_q_content or verse_word_tuples is empty.', None, [])
Processing  83 of 945
Error in prompt generation: ('Either p_q_content or verse_word_tuples is empty.', None, [])
Processing  84 of 945
Error in prompt generation: ('Either p_q_content or verse_word_tuples is empty.', None, [])
Processing  159 of 945
Error in prompt generation: ('Either p_q_content or verse_word_tuples is empty.', None, [])
Processing  160 of 945
Error in prompt generation: ('Either p_q_content or verse_word_tuples is empty.', None, [])
Processing  161 of 945
Error in prompt generation: ('Either p_q_content or verse_word_tuples is empty.', None, [])
Processing  258 of 945
Error in prompt generation: ('Either p_q_content or verse_word_tuples is empty.', None, [])
Processing  259 of 945
Error in prompt generation: ('Either p_q_content or verse_word_tuples is empty.', None, [])
Processing LUK 23:06 284 of 945
Rewriting verse LUK 23:06 to LUK 23:06
Error in pro

In [53]:
data = []
with open('output_Nov_14_nathan.jsonl', 'r', encoding='utf-8') as file:
    for line in file:
        data.append(json.loads(line))

In [54]:
def generate_evaluation_prompt(result):
    verse = result['vref']
    results = result['data']
    verse_df = ids_df[ids_df['b_ch_v'] == verse] 
    
    prompt = f'''
    ## Instruction
    Evaluate the data provided and compare it to the dataset. See if the "protasis" and "apodosis" fields contain all the necessary words from the table. Not all words from the table will be in the data since some words may not be a part of the protasis or apodosis.
    
    Here is an example of a passing protasis-apodosis analysis:

## Context

gloss
And
having
the
[one] tempting
he said
to Him
If
Son
You
-
of God
speak
that
the
stones
these
loaves of bread
might become

data = {{
    "protasis": [
        ('n40004003007', 'If', 'Εἰ'),
        ('n40004003008', 'Son', 'υἱὸς'), 
        ('n40004003009', 'You are', 'εἶ'), 
        ('n40004003010', '-', 'τοῦ'),
        ('n40004003011', 'of God', 'θεοῦ'),
    ]
    "apodosis": [
        ('n40004003012', 'speak', 'εἰπὲ'), 
        ('n40004003013', 'that', 'ἵνα'), 
        ('n40004003014', 'the', 'οἱ'), 
        ('n40004003015', 'stones', 'λίθοι'), 
        ('n40004003016', 'these', 'οὗτοι'), 
        ('n40004003017', 'loaves of bread', 'ἄρτοι'), 
        ('n40004003018', 'might become', 'γένωνται'),
    ]
}}
```

## Results

{{"result": "pass"}}

Here is an example of a failing protasis-apodosis analysis. Notice that a lot of words from the table are missed in the json.

## Context
gloss
you
are
the
salt
the
earth
if
but
the
salt
lost ~ taste
by
what
made salty
for
no ~ anything
good
longer
except
*
thrown
out
trampled under foot
by
NaN
men

```
{{
    "protasis": 
        [["n40005013007", "if", "ἐὰν"], 
        ["n40005013010", "the", "τὸ"], 
        ["n40005013011", "becomes tasteless", "μωρανθῇ"]], 
    "apodosis": 
        [["n40005013019", "if", "εἰ"], 
        ["n40005013020", "not", "μὴ"], 
        ["n40005013021", "having been cast", "βληθὲν"], 
        ["n40005013022", "out", "ἔξω"], 
        ["n40005013023", "to be trampled upon", "καταπατεῖσθαι"]]}}
```

## Results

{{"result": "fail"}}

Now follow a similar format with this protasis and apodosis pair using perfect JSON:

## Context 
{verse_df['gloss'].to_string(index=False)}

{results}

## Results:
'''
    return prompt

In [55]:
# define your GPT completion function
model = 'gpt-3.5-turbo' # 'gpt-4-1106-preview' #3.5-turbo

import ast

MAX_RETRIES = 10
def evaluate(prompt):
    system_prompt = "Analyze the p-q results and evaluate if they are correctly annotated. Note that some verses may have more than one p-q pair, so just use the words that match the complete wording. The other pairs will be handled in subsequent prompts."
    messages = [
        {"role": 'system', "content": system_prompt},
        {"role": 'user', 'content': prompt}
    ]
    for i in range(MAX_RETRIES):
        try:
            response = openai.ChatCompletion.create(
                model=model,
                messages=messages,
                temperature=0,
            )
            generated_texts = [
                choice.message["content"] for choice in response["choices"]
            ]
            result = ast.literal_eval(generated_texts[0])
            return result
        except Exception as e:
            print('Error in alignment:', e)
            if i < MAX_RETRIES - 1:  # i is zero indexed
                # time.sleep(1)
                continue
            else:
                return {"error": str(e)}

In [59]:
eval_results = []
for i, ref in enumerate(data):
    if ref['vref'] not in existing_refs:
        print('Processing', ref, i, 'of', len(data))
        
        try:
            prompt = generate_evaluation_prompt(ref)
        except Exception as e:
            print('Error in prompt generation:', e)
            continue
        
        if not prompt:
            print('Prompt is empty for', ref)
            pass
        results = evaluate(prompt)
        print(f'RESULTS = {results}')
        
        try:
            # extract the json from between the triple backticks
            if not type(results) == dict:
                raise ValueError('results is not a dict:', results)
            else:
                # store results
                eval_results.append((ref, results))
                print(ref, results)
                with open('pass_fail.jsonl', 'a', encoding='utf-8') as file:
                    json_output = json.dumps({"vref": ref, "results": results}, ensure_ascii=False)
                    file.write(json_output)
                    file.write('\n')
        except Exception as e:
            print('Error in result parsing:', e)

Processing {'vref': '1TH 3:8', 'data': {'protasis': [['n52003008004', 'if', 'ἐὰν'], ['n52003008005', 'you', 'ὑμεῖς'], ['n52003008006', 'stand firm', 'στήκετε'], ['n52003008007', 'in', 'ἐν'], ['n52003008008', 'Lord', 'κυρίῳ']], 'apodosis': [['n52003008001', 'for', 'ὅτι'], ['n52003008002', 'now', 'νῦν'], ['n52003008003', 'live', 'ζῶμεν']]}} 792 of 869
RESULTS = {'result': 'pass'}
{'vref': '1TH 3:8', 'data': {'protasis': [['n52003008004', 'if', 'ἐὰν'], ['n52003008005', 'you', 'ὑμεῖς'], ['n52003008006', 'stand firm', 'στήκετε'], ['n52003008007', 'in', 'ἐν'], ['n52003008008', 'Lord', 'κυρίῳ']], 'apodosis': [['n52003008001', 'for', 'ὅτι'], ['n52003008002', 'now', 'νῦν'], ['n52003008003', 'live', 'ζῶμεν']]}} {'result': 'pass'}
Processing {'vref': '1TH 4:14', 'data': {'protasis': [['n52004014001', 'if', 'εἰ'], ['n52004014003', 'we believe', 'πιστεύομεν'], ['n52004014004', 'that', 'ὅτι'], ['n52004014005', 'Jesus', 'Ἰησοῦς'], ['n52004014006', 'died', 'ἀπέθανεν'], ['n52004014007', 'and', 'καὶ'], 

In [60]:
# check to see if every b_ch_v in mark_df is in the output file
result_list_from_file = []
vrefs_for_lines_with_errors = []
with open('output_Nov_14_nathan.jsonl', 'r', encoding='utf-8') as file: 
    for line in file:
        line_json = json.loads(line)
        result_list_from_file.append(line_json['vref'])
        
        if 'error' in line:
            vrefs_for_lines_with_errors.append(line_json['vref'])


# filter mark_df to remove rows where b_ch_v is NaN or empty string
mark_df = mark_df[mark_df['b_ch_v'].notna()]

print(len(result_list_from_file), 'of', len(mark_df), 'verses processed.')

missing_verses = []
for i, row in mark_df.iterrows():
    if not row['b_ch_v']:
        continue
    elif row['b_ch_v'] not in result_list_from_file:
        print(i, row['b_ch_v'], 'not in result_list')
        missing_verses.append(row['b_ch_v'])
        
    

869 of 945 verses processed.
284 LUK 23:06 not in result_list
715 GAL 6:9 not in result_list
801 Titus 1:6 not in result_list


In [61]:
missing_verse_dict = {
"LUK 23:06": "LUK 23:06",
"GAL 6:9": "GAL 6:9",
"1Th 3:8": "1TH 3:8",
"1Th 4:14": "1TH 4:14",
"2Th 2:3": "2TH 2:3",
"2Th 3:10": "2TH 3:10",
"2Th 3:14": "2TH 3:14",
"1Th 2:7": "1TH 2:7",
"1Th 5:10": "1TH 5:10",
"2Th 1:5": "2TH 1:5",
"2Th 2:15": "2TH 2:15",
"Tim 1:8": "1TI 1:8",
"Tim 2:15": "1TI 2:15",
"Tim 3:1": "1TI 3:1",
"Tim 3:5": "1TI 3:5",
"Tim 3:15": "1TI 3:15",
"Tim 5:4": "1TI 5:4",
"Tim 5:8": "1TI 5:8",
"Tim 5:9": "1TI 5:9",
"Timothy 5:16": "1TI 5:16",
"Timothy 5:19": "1TI 5:19",
"Tim 6:3": "1TI 6:3",
"Tim 1:10": "2TI 1:10",
"Tim 2:5": "2TI 2:5",
"Tim 2:11": "2TI 2:11",
"Tim 2:12": "2TI 2:12",
"Tim 2:12": "2TI 2:12",
"Tim 2:13": "2TI 2:13",
"Tim 2:21": "2TI 2:21",
"Titus 1:6": "TIT 1:6",
"Peter 1:6": "1PE 1:6",
"Peter 1:17": "1PE 1:17",
"Peter 2:2": "1PE 2:2",
"Peter 2:19": "1PE 2:19",
"Peter 2:20": "1PE 2:20",
"Peter 2:20": "1PE 2:20",
"Peter 3:1": "1PE 3:1",
"Peter 3:13": "1PE 3:13",
"Peter 3:14": "1PE 3:14",
"Peter 3:17": "1PE 3:17",
"Peter 4:11": "1PE 4:11",
"Peter 4:11": "1PE 4:11",
"Peter 4:14": "1PE 4:14",
"Peter 4:16": "1PE 4:16",
"Peter 4:17": "1PE 4:17",
"Peter 4:18": "1PE 4:18",
"Peter 2:4": "2PE 2:4",
"Peter 2:20": "2PE 2:20",
"Peter 2:13": "2PE 2:13",
"John 1:6": "1JN 1:6",
"John 1:7": "1JN 1:7",
"John 1:8": "1JN 1:8",
"John 1:9": "1JN 1:9",
"John 1:10": "1JN 1:10",
"John 2:1": "1JN 2:1",
"John 2:3": "1JN 2:3",
"John 2:15": "1JN 2:15",
"John 2:19": "1JN 2:19",
"John 2:24": "1JN 2:24",
"John 2:29": "1JN 2:29",
"John 3:13": "1JN 3:13",
"John 3:20": "1JN 3:20",
"John 3:21": "1JN 3:21",
"John 4:11": "1JN 4:11",
"John 4:12": "1JN 4:12",
"John 4:20": "1JN 4:20",
"John 5:9": "1JN 5:9",
"John 5:14": "1JN 5:14",
"John 5:15": "1JN 5:15",
"John 5:16": "1JN 5:16",
"John 2:22": "1JN 2:22",
"John 2:28": "1JN 2:28",
"John 3:2": "1JN 3:2",
"John 3:22": "1JN 3:22",
"John 4:1": "1JN 4:1",
"John 4:15": "1JN 4:15",
"John 5:5": "1JN 5:5",
"John 5:15": "1JN 5:15",
}

In [63]:
from tqdm import tqdm
# add each result by vref to the mark_df where 'ref' == 'b_ch_v'
mark_df['matched_protasis_words'] = [[] for _ in range(len(mark_df))]
mark_df['matched_apodosis_words'] = [[] for _ in range(len(mark_df))]

with open('output_Nov_14_nathan.jsonl', 'r', encoding='utf-8') as file:
    # format: json_output = json.dumps({"vref": ref, "data": results}, ensure_ascii=False)
    for line in tqdm(file):
        result = json.loads(line)
        vref = result['vref']
        if vref in vrefs_for_lines_with_errors:
            pass
        try:
            data = result['data']
            protasis = data.get('protasis', [])
            apodosis = data.get('apodosis', [])
            for i, row in mark_df.iterrows():
                if row['b_ch_v'] == vref:
                    mark_df.at[i, 'matched_protasis_words'] = protasis
                    mark_df.at[i, 'matched_apodosis_words'] = apodosis
                    break
        except KeyError:
            print('KeyError in', result)
            continue
        

869it [00:16, 51.56it/s] 


In [ ]:
mark_df.to_csv('FINAL_all_conditionals_with_verse_data_and_alignment.tsv', sep='\t', index=False)

In [64]:
mark_df.head()

,reference,class,inv.,probability,time orientation,illocutionary force,english translations,notes,parallel passage(s),unnamed: 10,unnamed:,parallel passages,unnamed: 9,b_ch_v,scope,protasis,apodosis,matched_protasis_words,matched_apodosis_words,all_verse_word_tuples
0,MAT 4:3,1,,Factual,Present,Exhort,"ESV, NASB, NRSV, NIV, NLT: ""if""","P presents a fact that the temptor knew to be true. The devil certainly knows Jesus' identity. Hagner (1993) notes that this might as well be translated ""since"", as the devil is testing Jesus' obedience to the Father rather than questioning his identity. Q presents the subsequent command Jesus is being tempted to obey. The conditional as a whole amounts to a taunt or a challenge. Fong (2014: 30-31) discusses this as an example given by Young (1994) of a rhetorical conditional. That is, it is not being presented as a real condition, but the conditional is used for its logical connection in order to affect some other speech act, in this case ""manipulation"", according to Young. Fong, however, thinks it would be better classified as a ""challenge.""",Luke 4:3,,,,,MAT 4:3,p: (if you are the Son of God)\nq: (command these stones to become bread),[],[],"[[n40004003007, if, Εἰ], [n40004003008, son, υἱὸς], [n40004003009, are, εἶ], [n40004003010, NaN, τοῦ], [n40004003011, God, θεοῦ]]","[[n40004003012, command, εἰπὲ], [n40004003013, to, ἵνα], [n40004003014, NaN, οἱ], [n40004003015, stones, λίθοι], [n40004003016, these, οὗτοι], [n40004003017, bread, ἄρτοι], [n40004003018, become, γένωνται]]","[('n40004003001', 'And', 'καὶ'), ('n40004003002', 'having come', 'προσελθὼν'), ('n40004003003', 'the', 'ὁ'), ('n40004003004', '[one] tempting', 'πειράζων'), ('n40004003005', 'he said', 'εἶπεν'), ('n40004003006', 'to Him', 'αὐτῷ'), ('n40004003007', 'If', 'Εἰ'), ('n40004003008', 'Son', 'υἱὸς'), ('n40004003009', 'You are', 'εἶ'), ('n40004003010', '-', 'τοῦ'), ('n40004003011', 'of God', 'θεοῦ'), ('n40004003012', 'speak', 'εἰπὲ'), ('n40004003013', 'that', 'ἵνα'), ('n40004003014', 'the', 'οἱ'), ('n40004003015', 'stones', 'λίθοι'), ('n40004003016', 'these', 'οὗτοι'), ('n40004003017', 'loaves of bread', 'ἄρτοι'), ('n40004003018', 'might become', 'γένωνται')]"
1,MAT 4:6,1,,Factual,Present,Exhort,"ESV, NASB, NRSV, NIV, NLT: ""if""","As with 4:3, this conditional expresses no doubt about Jesus' identity, but is being used to manipulate, tempt, and challenge him.",Luke 4:9,,,,,MAT 4:6,p: (if you are the Son of God)\nq: (throw yourself down),[],[],"[[n40004006004, if, Εἰ], [n40004006005, son, υἱὸς], [n40004006006, are, εἶ], [n40004006007, -, τοῦ], [n40004006008, God, θεοῦ]]","[[n40004006009, throw, βάλε], [n40004006010, yourself, σεαυτὸν], [n40004006011, down, κάτω]]","[('n40004006001', 'and', 'καὶ'), ('n40004006002', 'says', 'λέγει'), ('n40004006003', 'to him', 'αὐτῷ'), ('n40004006004', 'If', 'Εἰ'), ('n40004006005', 'Son', 'υἱὸς'), ('n40004006006', 'You are', 'εἶ'), ('n40004006007', '-', 'τοῦ'), ('n40004006008', 'of God', 'θεοῦ'), ('n40004006009', 'throw', 'βάλε'), ('n40004006010', 'Yourself', 'σεαυτὸν'), ('n40004006011', 'down', 'κάτω'), ('n40004006012', 'it has been written', 'γέγραπται'), ('n40004006013', 'for', 'γὰρ'), ('n40004006014', '-', 'ὅτι'), ('n40004006015', 'To the', 'Τοῖς'), ('n40004006016', 'angels', 'ἀγγέλοις'), ('n40004006017', 'of Him', 'αὐτοῦ'), ('n40004006018', 'He will give orders', 'ἐντελεῖται'), ('n40004006019', 'concerning', 'περὶ'), ('n40004006020', 'You', 'σοῦ'), ('n40004006021', 'and', 'καὶ'), ('n40004006022', 'in', 'ἐπὶ'), ('n40004006023', '[their] hands', 'χειρῶν'), ('n40004006024', 'will they bear up', 'ἀροῦσίν'), ('n40004006025', 'You', 'σε'), ('n40004006026', '', 'μήποτε'), ('n40004006027', 'You strike', 'προσκόψῃς'), ('n40004006028', 'against', 'πρὸς'), ('n40004006029', 'a stone', 'λίθον'), ('n40004006030', 'the', 'τὸν'), ('n40004006031', 'foot', 'πόδα'), ('n40004006032', 'of You', 'σου')]"
2,MAT 4:9,3,x,Very Unlikely,Present,Promise / Exhort,"ESV, NASB, NRSV, NIV, NLT: ""if""","In 4:3 and 4:6

In [65]:
output_df_copy = mark_df.copy()

# filter these columns:
columns_to_filter = [
    'unnamed: 10',
    'unnamed:',
    'unnamed: 9',
    'macula_ids_in_protasis',
    'macula_ids_in_apodosis',
    'matched_english_in_protasis',
    'matched_english_in_apodosis',
    'matched_gloss_in_protasis',
    'matched_gloss_in_apodosis',
    'all_verse_word_tuples',
    'matches_map',
    'all_protasis_words',
    'all_apodosis_words',
    'unmatched_protasis_words',
    'unmatched_apodosis_words',
    'protasis',
    'apodosis',
]

columns_to_filter = [column for column in columns_to_filter if column in output_df_copy.columns]

output_df_copy.drop(columns=columns_to_filter, inplace=True)
# rename matched_protasis_words -> protasis	matched_apodosis_words -> apodosis
output_df_copy.rename(columns={'matched_protasis_words': 'protasis', 'matched_apodosis_words': 'apodosis'}, inplace=True)
# output_df_copy.head()

In [66]:
output_df_copy.head()

,reference,class,inv.,probability,time orientation,illocutionary force,english translations,notes,parallel passage(s),parallel passages,b_ch_v,scope,protasis,apodosis
0,MAT 4:3,1,,Factual,Present,Exhort,"ESV, NASB, NRSV, NIV, NLT: ""if""","P presents a fact that the temptor knew to be true. The devil certainly knows Jesus' identity. Hagner (1993) notes that this might as well be translated ""since"", as the devil is testing Jesus' obedience to the Father rather than questioning his identity. Q presents the subsequent command Jesus is being tempted to obey. The conditional as a whole amounts to a taunt or a challenge. Fong (2014: 30-31) discusses this as an example given by Young (1994) of a rhetorical conditional. That is, it is not being presented as a real condition, but the conditional is used for its logical connection in order to affect some other speech act, in this case ""manipulation"", according to Young. Fong, however, thinks it would be better classified as a ""challenge.""",Luke 4:3,,MAT 4:3,p: (if you are the Son of God)\nq: (command these stones to become bread),"[[n40004003007, if, Εἰ], [n40004003008, son, υἱὸς], [n40004003009, are, εἶ], [n40004003010, NaN, τοῦ], [n40004003011, God, θεοῦ]]","[[n40004003012, command, εἰπὲ], [n40004003013, to, ἵνα], [n40004003014, NaN, οἱ], [n40004003015, stones, λίθοι], [n40004003016, these, οὗτοι], [n40004003017, bread, ἄρτοι], [n40004003018, become, γένωνται]]"
1,MAT 4:6,1,,Factual,Present,Exhort,"ESV, NASB, NRSV, NIV, NLT: ""if""","As with 4:3, this conditional expresses no doubt about Jesus' identity, but is being used to manipulate, tempt, and challenge him.",Luke 4:9,,MAT 4:6,p: (if you are the Son of God)\nq: (throw yourself down),"[[n40004006004, if, Εἰ], [n40004006005, son, υἱὸς], [n40004006006, are, εἶ], [n40004006007, -, τοῦ], [n40004006008, God, θεοῦ]]","[[n40004006009, throw, βάλε], [n40004006010, yourself, σεαυτὸν], [n40004006011, down, κάτω]]"
2,MAT 4:9,3,x,Very Unlikely,Present,Promise / Exhort,"ESV, NASB, NRSV, NIV, NLT: ""if""","In 4:3 and 4:6 the exhortations (or temptations) are direct and occur in q; in this verse the exhortation (or temptation) is made indirectly in p. Though highly unlikely, and though it did not come to pass, this was still a (remote) possibility when the devil spoke it. It was a real option for Jesus, though he did not take it. Unlike the two previous temptations, this one is a third class, using εαν with the subjunctive because ( according to Hagner 1993) it involves an actual unmet condition.",Luke 4:7,,MAT 4:9,q:(All these I will give you)\np: (if you will fall down and worship me),"[[n40004009008, if, ἐὰν], [n40004009009, fall down, πεσὼν], [n40004009010, worship, προσκυνήσῃς], [n40004009003, to Him, αὐτῷ]]","[[n40004009007, will I give, δώσω], [n40004009004, These things, Ταῦτά], [n40004009006, all, πάντα], [n40004009005, to You, σοι], [n40004009011, me, μοι]]"
3,MAT 5:13a,3,,Unlikely,Gnomic,Assert,"ESV, NASB, NRSV, NIV, NLT: ""if""","This conditional must be read in its context of describing discipleship. The connection between salt and discipleship is that, as Marshall (1978) puts it, ""a false form of discipleship may look like salt, but the gradual process of leeching leaves only a zestless pile of waste."" Nolland (2005) claims that discussions of how salt of the day may have lost its flavour are pointless, as the point being made is that such a thing would be ""bizarre and unnatural."" It would be terrible for salt to lose saltiness because it is itself the thing that adds flavour to tasteless food.",Mark 9:50; Luke 14:34,,MAT 5:13,p: (if salt has lost its taste)\nq: (how shall its saltiness be restored?),"[[n40005013011, the, the], [n40005013012, salt, salt], [n40005013013, lost ~ taste, becomes tasteless]]","[[n40005013020, *, not], [n40005013021, thrown, having been cast], [n40005013022, out, out], [n40005013023, trampled under foot, to be trampled upon]]"
4,MAT 5:20,3,,Very Unlikely,Gnomic,Warn,"ESV, NASB, NRSV, NIV, NLT: "

In [ ]:
output_df_copy.to_html('output.html')
output_df_copy.to_csv('output.tsv', sep='\t', index=False)


In [67]:
with open('pass_fail.jsonl', 'r', encoding='utf-8') as file:
    gloss_data = file.readlines()

In [68]:
import pandas as pd
import jsonlines
import numpy as np

# Open the JSONL file
with jsonlines.open('pass_fail.jsonl') as reader:
    # Initialize empty lists for each column
    vref = []
    protasis = []
    apodosis = []
    data_result = []
    results_result = []

    # Iterate over each line in the JSONL file
    for line in reader:
        # Append the values to the respective lists
        vref.append(line['vref']['vref'])
        try:
            protasis.append(line['vref']['data']['protasis'])
        except:
            protasis.append('NaN')
        apodosis.append(line['vref']['data'].get('apodosis'))
        try:
            data_result.append(line['data']['result'])
        except:
            data_result.append('NaN')
        try:
            results_result.append(line['results']['result'])
        except:
            results_result.append('NaN')

# Replace "NaN" strings with actual NaN values
data_result = [np.nan if val == "NaN" else val for val in data_result]
results_result = [np.nan if val == "NaN" else val for val in results_result]

print(len(vref))
print(len(protasis))
print(len(apodosis))
print(len(data_result))
print(data_result[-5:])
print(len(results_result))
print(results_result[-5:])
# Create a dataframe from the lists
df = pd.DataFrame({
    'vref.vref': vref,
    'vref.data.protasis': protasis,
    'vref.data.apodosis': apodosis,
    'data.result': data_result,
    'results.result': results_result
})

df['result'] = df['data.result'].fillna(df['results.result'])

# Drop the 'data.result' and 'results.result' columns
df.drop(['data.result', 'results.result'], axis=1, inplace=True)

df = df.rename(columns={'vref.vref': 'vref', 'vref.data.protasis': 'protasis', 'vref.data.apodosis': 'apodosis'})
# Print the dataframe
df.tail()

920
920
920
920
[nan, nan, nan, nan, nan]
920
['fail', 'fail', 'pass', 'pass', 'pass']


,vref,protasis,apodosis,result
915,1JN 3:22,"[[n62003022001, and, καὶ], [n62003022002, whatever, ὃ], [n62003022003, *, ἐὰν], [n62003022004, ask, αἰτῶμεν]]","[[n62003022005, receive, λαμβάνομεν], [n62003022006, from, ἀπ’], [n62003022007, him, αὐτοῦ]]",fail
916,1JN 4:1,"[[n62004001001, Beloved, Ἀγαπητοί], [n62004001002, not, μὴ], [n62004001003, every, παντὶ], [n62004001004, spirit, πνεύματι], [n62004001005, believe, πιστεύετε], [n62004001006, but, ἀλλὰ], [n62004001007, test, δοκιμάζετε], [n62004001008, the, τὰ], [n62004001009, spirits, πνεύματα], [n62004001010, whether, εἰ], [n62004001011, from, ἐκ], [n62004001012, -, τοῦ], [n62004001013, God, θεοῦ]]",[],fail
917,1JN 4:15,"[[n62004015001, anyone, ὃς], [n62004015002, if, ἐὰν], [n62004015003, confesses, ὁμολογήσῃ], [n62004015004, that, ὅτι], [n62004015005, Jesus, Ἰησοῦς], [n62004015006, is, ἐστιν], [n62004015007, the, ὁ], [n62004015008, Son, υἱὸς], [n62004015009, -, τοῦ], [n62004015010, God, θεοῦ]]","[[n62004015013, in, ἐν], [n62004015014, him, αὐτῷ], [n62004015015, abides, μένει], [n62004015016, and, καὶ], [n62004015017, he, αὐτὸς], [n62004015018, in, ἐν], [n62004015019, -, τῷ], [n62004015020, God, θεῷ]]",pass
918,1JN 5:5,"[[n62005005008, except, εἰ], [n62005005009, not, μὴ], [n62005005010, he, ὁ], [n62005005011, believes, πιστεύων], [n62005005012, that, ὅτι], [n62005005013, Jesus, Ἰησοῦς], [n62005005014, is, ἐστιν], [n62005005015, the, ὁ], [n62005005016, Son, υἱὸς], [n62005005017, -, τοῦ], [n62005005018, of God, θεοῦ]]","[[n62005005001, who, τίς], [n62005005002, now, δέ], [n62005005003, is, ἐστιν], [n62005005004, the, ὁ], [n62005005005, overcomes, νικῶν], [n62005005006, the, τὸν], [n62005005007, world, κόσμον]]",pass
919,1JN 5:15,"[[n62005015003, we know, οἴδαμεν], [n62005015005, He hears, ἀκούει], [n62005015006, us, ἡμῶν], [n62005015007, whatever, ὃ], [n62005015009, we might ask, αἰτώμεθα]]","[[n62005015010, we know, οἴδαμεν], [n62005015012, we have, ἔχομεν], [n62005015013, the, τὰ], [n62005015014, requests, αἰτήματα], [n62005015016, we have asked, ᾐτήκαμεν], [n62005015017, from, ἀπ’], [n62005015018, him, αὐτοῦ]]",pass


In [69]:
# Create a new column 'ref' in df that contains the 'vref' values without the trailing characters after the numbers
df['ref'] = df['vref'].str.extract(r'([1-3]*[A-Z]+\s\d+:\d+)', expand=False)

# Create a new column 'ref' in concatenated_df that contains the 'reference' values without the trailing characters after the numbers
mark_df['ref'] = mark_df['reference'].str.extract(r'([1-3]*[A-Z]+\s\d+:\d+)', expand=False)

# Merge df and concatenated_df on 'ref'
merged_df = pd.merge(df, mark_df[['ref', 'scope']], on='ref', how='left')

# Drop 'data.error', 'error', and 'reference' columns
merged_df = merged_df.drop(columns=['ref'])

In [70]:
merged_df = merged_df.rename(columns={'scope': 'p_q'})
merged_df.head()

,vref,protasis,apodosis,result,p_q
0,MAT 4:3,"[[n40004003007, If, Εἰ], [n40004003008, Son, υἱὸς], [n40004003009, You are, εἶ], [n40004003010, -, τοῦ], [n40004003011, of God, θεοῦ]]","[[n40004003012, speak, εἰπὲ], [n40004003013, that, ἵνα], [n40004003014, the, οἱ], [n40004003015, stones, λίθοι], [n40004003016, these, οὗτοι], [n40004003017, loaves of bread, ἄρτοι], [n40004003018, might become, γένωνται]]",pass,p: (if you are the Son of God)\nq: (command these stones to become bread)
1,MAT 4:6,"[[n40004006004, If, Εἰ], [n40004006005, Son, υἱὸς], [n40004006006, You are, εἶ], [n40004006007, -, τοῦ], [n40004006008, of God, θεοῦ]]","[[n40004006009, throw, βάλε], [n40004006010, Yourself, σεαυτὸν], [n40004006011, down, κάτω]]",pass,p: (if you are the Son of God)\nq: (throw yourself down)
2,MAT 4:9,"[[n40004009008, if, ἐὰν], [n40004009009, falling down, πεσὼν], [n40004009010, You will worship, προσκυνήσῃς], [n40004009011, me, μοι]]","[[n40004009004, These things, Ταῦτά], [n40004009005, to You, σοι], [n40004009006, all, πάντα], [n40004009007, will I give, δώσω]]",pass,q:(All these I will give you)\np: (if you will fall down and worship me)
3,MAT 5:13,"[[n40005013007, if, ἐὰν], [n40005013010, the, τὸ], [n40005013011, becomes tasteless, μωρανθῇ]]","[[n40005013019, if, εἰ], [n40005013020, not, μὴ], [n40005013021, having been cast, βληθὲν], [n40005013022, out, ἔξω], [n40005013023, to be trampled upon, καταπατεῖσθαι]]",fail,p: (if salt has lost its taste)\nq: (how shall its saltiness be restored?)
4,MAT 5:13,"[[n40005013007, if, ἐὰν], [n40005013010, the, τὸ], [n40005013011, becomes tasteless, μωρανθῇ]]","[[n40005013019, if, εἰ], [n40005013020, not, μὴ], [n40005013021, having been cast, βληθὲν], [n40005013022, out, ἔξω], [n40005013023, to be trampled upon, καταπατεῖσθαι]]",fail,q: (it is no longer good for anything)\np: (except to be thrown out and trampled under people's feet)


In [71]:
# Use regular expressions to extract 'p' and 'q' data
merged_df['p'] = merged_df['p_q'].str.extract(r'p:\s*\((.*?)\)', expand=False)
merged_df['q'] = merged_df['p_q'].str.extract(r'q:\s*\((.*?)\)', expand=False)

# Fill NaN values in both 'p' and 'q' columns with NaN
merged_df['p'].fillna("NaN", inplace=True)
merged_df['q'].fillna("NaN", inplace=True)

# Drop the original column
merged_df.drop('p_q', axis=1, inplace=True)

In [72]:
merged_df = merged_df.rename(columns={'protasis': 'Protasis Gloss'})
merged_df = merged_df.rename(columns={'apodosis': 'Apodosis Gloss'})
merged_df = merged_df.rename(columns={'p': 'Protasis'})
merged_df = merged_df.rename(columns={'q': 'Apodosis'})

In [73]:
merged_df = merged_df[['vref', 'Protasis', 'Protasis Gloss', 'Apodosis', 'Apodosis Gloss','result']]

In [74]:
final_df = merged_df.copy()
final_df.head()

,vref,Protasis,Protasis Gloss,Apodosis,Apodosis Gloss,result
0,MAT 4:3,if you are the Son of God,"[[n40004003007, If, Εἰ], [n40004003008, Son, υἱὸς], [n40004003009, You are, εἶ], [n40004003010, -, τοῦ], [n40004003011, of God, θεοῦ]]",command these stones to become bread,"[[n40004003012, speak, εἰπὲ], [n40004003013, that, ἵνα], [n40004003014, the, οἱ], [n40004003015, stones, λίθοι], [n40004003016, these, οὗτοι], [n40004003017, loaves of bread, ἄρτοι], [n40004003018, might become, γένωνται]]",pass
1,MAT 4:6,if you are the Son of God,"[[n40004006004, If, Εἰ], [n40004006005, Son, υἱὸς], [n40004006006, You are, εἶ], [n40004006007, -, τοῦ], [n40004006008, of God, θεοῦ]]",throw yourself down,"[[n40004006009, throw, βάλε], [n40004006010, Yourself, σεαυτὸν], [n40004006011, down, κάτω]]",pass
2,MAT 4:9,if you will fall down and worship me,"[[n40004009008, if, ἐὰν], [n40004009009, falling down, πεσὼν], [n40004009010, You will worship, προσκυνήσῃς], [n40004009011, me, μοι]]",All these I will give you,"[[n40004009004, These things, Ταῦτά], [n40004009005, to You, σοι], [n40004009006, all, πάντα], [n40004009007, will I give, δώσω]]",pass
3,MAT 5:13,if salt has lost its taste,"[[n40005013007, if, ἐὰν], [n40005013010, the, τὸ], [n40005013011, becomes tasteless, μωρανθῇ]]",how shall its saltiness be restored?,"[[n40005013019, if, εἰ], [n40005013020, not, μὴ], [n40005013021, having been cast, βληθὲν], [n40005013022, out, ἔξω], [n40005013023, to be trampled upon, καταπατεῖσθαι]]",fail
4,MAT 5:13,except to be thrown out and trampled under people's feet,"[[n40005013007, if, ἐὰν], [n40005013010, the, τὸ], [n40005013011, becomes tasteless, μωρανθῇ]]",it is no longer good for anything,"[[n40005013019, if, εἰ], [n40005013020, not, μὴ], [n40005013021, having been cast, βληθὲν], [n40005013022, out, ἔξω], [n40005013023, to be trampled upon, καταπατεῖσθαι]]",fail


In [75]:
protasis_data = final_df['Protasis Gloss'].tolist()
apodosis_data = final_df['Apodosis Gloss'].tolist()

In [76]:
def generate_lists(list_data):
    ids = []
    gloss = []
    gr = []

    for lst in list_data:
        vr_id = []
        vr_gl = []
        vr_gr = []
        
        try:
            for i in lst:
                vr_id.append(i[0])
                vr_gl.append(i[1])
                vr_gr.append(i[2])
        except:
            vr_id.append('')
            vr_gl.append('')
            vr_gr.append('')
        
        ids.append(vr_id)
        gloss.append(vr_gl)
        gr.append(vr_gr)
        
    return ids, gloss, gr

In [77]:
prot_ids, prot_gloss, prot_gr = generate_lists(protasis_data)
apo_ids, apo_gloss, apo_gr = generate_lists(apodosis_data)

In [78]:
final_df['Protasis IDs'] = prot_ids
final_df['Protasis Glosses'] = prot_gloss
final_df['Protasis Greek'] = prot_gr
final_df['Apodosis IDs'] = apo_ids
final_df['Apodosis Glosses'] = apo_gloss
final_df['Apodosis Greek'] = apo_gr

In [79]:
final_df = final_df[['vref', 'Protasis', 'Protasis IDs', 'Protasis Glosses', 'Protasis Greek', 'Apodosis', 'Apodosis IDs', 'Apodosis Glosses', 'Apodosis Greek', 'result']]
final_df.head()

,vref,Protasis,Protasis IDs,Protasis Glosses,Protasis Greek,Apodosis,Apodosis IDs,Apodosis Glosses,Apodosis Greek,result
0,MAT 4:3,if you are the Son of God,"[n40004003007, n40004003008, n40004003009, n40004003010, n40004003011]","[If, Son, You are, -, of God]","[Εἰ, υἱὸς, εἶ, τοῦ, θεοῦ]",command these stones to become bread,"[n40004003012, n40004003013, n40004003014, n40004003015, n40004003016, n40004003017, n40004003018]","[speak, that, the, stones, these, loaves of bread, might become]","[εἰπὲ, ἵνα, οἱ, λίθοι, οὗτοι, ἄρτοι, γένωνται]",pass
1,MAT 4:6,if you are the Son of God,"[n40004006004, n40004006005, n40004006006, n40004006007, n40004006008]","[If, Son, You are, -, of God]","[Εἰ, υἱὸς, εἶ, τοῦ, θεοῦ]",throw yourself down,"[n40004006009, n40004006010, n40004006011]","[throw, Yourself, down]","[βάλε, σεαυτὸν, κάτω]",pass
2,MAT 4:9,if you will fall down and worship me,"[n40004009008, n40004009009, n40004009010, n40004009011]","[if, falling down, You will worship, me]","[ἐὰν, πεσὼν, προσκυνήσῃς, μοι]",All these I will give you,"[n40004009004, n40004009005, n40004009006, n40004009007]","[These things, to You, all, will I give]","[Ταῦτά, σοι, πάντα, δώσω]",pass
3,MAT 5:13,if salt has lost its taste,"[n40005013007, n40005013010, n40005013011]","[if, the, becomes tasteless]","[ἐὰν, τὸ, μωρανθῇ]",how shall its saltiness be restored?,"[n40005013019, n40005013020, n40005013021, n40005013022, n40005013023]","[if, not, having been cast, out, to be trampled upon]","[εἰ, μὴ, βληθὲν, ἔξω, καταπατεῖσθαι]",fail
4,MAT 5:13,except to be thrown out and trampled under people's feet,"[n40005013007, n40005013010, n40005013011]","[if, the, becomes tasteless]","[ἐὰν, τὸ, μωρανθῇ]",it is no longer good for anything,"[n40005013019, n40005013020, n40005013021, n40005013022, n40005013023]","[if, not, having been cast, out, to be trampled upon]","[εἰ, μὴ, βληθὲν, ἔξω, καταπατεῖσθαι]",fail


In [80]:
def highlight_results(val):
    color = 'green' if val == 'pass' else 'red'
    return f'background-color: {color}'

styled_table = final_df.style.applymap(highlight_results, subset=['result']).to_html(index=False)

with open('protasis_apodosis_table3.html', 'a', encoding='utf-8') as file:
    file.write(styled_table)

## Use syntax for unit resolution

In [ ]:
def process_element(element, usfm_ref, indent=0, brackets=True):
    # if brackets:
    #     indent = 0
    treedown_str = ""
    open_bracket = "[" if brackets else ""
    close_bracket = "] " if brackets else ""

    if element.get("class") == "cl":
        treedown_str += "\n" + ("  " * indent) + open_bracket

    if element.get("role"):
        role = element.attrib["role"]
        if role == "adv":
            role = "+"
        # if not brackets:
        treedown_str += "\n"
        treedown_str += ("  " * indent) + open_bracket + role + ": "

    # # bold the matching token using usfm ref # NOTE: not applicable, since I think you have to use a USFM ref without the word on the endpoint
    # if element.tag == "w" and element.get("ref") == usfm_ref:
    #     treedown_str += "**" + element.text + "**"
    #     treedown_str += element.attrib.get("after", "") + close_bracket

    if element.tag == "w" and element.text:
        treedown_str += str(
            (element.text, element.attrib.get("gloss", ""), element.attrib.get("id", ""))
        )
        treedown_str += close_bracket

    for child in element:
        treedown_str += process_element(child, usfm_ref, indent + 1, brackets)

    return treedown_str


def get_treedown_by_ref(usfm_ref, brackets=False):
    usfm_passage = usfm_ref.split("!")[0]
    endpoint = (
        "https://labs.clear.bible/symphony-dev/api/GNT/Nestle1904/lowfat?usfm-ref="
        # "http://localhost:8984/symphony-dev/api/GNT/Nestle1904/lowfat?usfm-ref="
        + usfm_passage
    )

    # uri encode endpoint
    endpoint = requests.utils.requote_uri(endpoint)

    # print(endpoint)

    text_response = requests.get(endpoint).text
    xml = etree.fromstring(text_response.encode("utf-8"))

    treedown = process_element(xml, usfm_passage, brackets=brackets)
    return treedown

# test
print(get_treedown_by_ref("MAT 6:23", brackets=True))

In [ ]:


# Assuming mark_df is your dataframe and it's already loaded in your notebook.
# If not, you need to load it here.

# Function definitions (e.g., get_treedown_by_ref, process_element) should already be defined.
def get_treedown_by_ref(usfm_ref, brackets=False):
    usfm_passage = usfm_ref.split("!")[0]
    endpoint = (
        "https://labs.clear.bible/symphony-dev/api/GNT/Nestle1904/lowfat?usfm-ref="
        # "http://localhost:8984/symphony-dev/api/GNT/Nestle1904/lowfat?usfm-ref="
        + usfm_passage
    )

    # uri encode endpoint
    endpoint = requests.utils.requote_uri(endpoint)

    # print(endpoint)

    try:
        text_response = requests.get(endpoint).text
        xml = etree.fromstring(text_response.encode("utf-8"))
    except Exception as e:
        print(f"An error occurred: {e}")
        xml = None
    return xml
    # treedown = process_element(xml, usfm_passage, brackets=brackets)
    # return treedown


def get_siblings(element):
    # This function returns all sibling elements
    return [e for e in element.getparent()]

def get_word_info(element):
    # This function extracts the text, gloss, and id from a word element
    return element.text, element.attrib.get("gloss", ""), element.attrib.get("id", "")

def update_matched_words(matched_words, siblings):
    # This function updates the matched words list with any new matches found in siblings
    updated_matches = matched_words.copy()
    for sibling in siblings:
        if sibling.tag == "w":
            word_info = get_word_info(sibling)
            if word_info not in updated_matches:
                updated_matches.append(word_info)
    return updated_matches

In [ ]:
# Main processing loop
for index, row in tqdm(mark_df.iterrows()):
    ref = row['b_ch_v']
    if not ref:
        continue
    xml_root = get_treedown_by_ref(ref)  # Get the XML tree for the reference
    
    if not xml_root:
        continue
    # xml_root = etree.fromstring(treedown_xml.encode('utf-8'))  # Parse the XML string to an ElementTree object

    # Extract matched words lists from the dataframe
    matched_protasis_words = row['matched_protasis_words']
    matched_apodosis_words = row['matched_apodosis_words']

    # Find all word elements
    word_elements = xml_root.findall(".//w")

    # Iterate over matched words to find siblings in the XML tree
    for word_tuple in matched_protasis_words + matched_apodosis_words:
        if type(word_tuple) == dict:
            word_id = word_tuple['id']  # Get the id of the word
        else:
            word_id = word_tuple[2]
        # Find the corresponding element in the XML tree
        word_element = xml_root.xpath(f".//*[@id='{word_id}']")
        if word_element:
            siblings = get_siblings(word_element[0])  # Get siblings of the found element
            if word_tuple in matched_protasis_words:
                # Update matched_protasis_words if the word is in protasis
                mark_df.at[index, 'matched_protasis_words'] = update_matched_words(matched_protasis_words, siblings)
            else:
                # Update matched_apodosis_words if the word is in apodosis
                mark_df.at[index, 'matched_apodosis_words'] = update_matched_words(matched_apodosis_words, siblings)



In [ ]:
mark_df.head()

In [ ]:
output_df_copy = mark_df.copy()

# filter these columns:
columns_to_filter = [
    'unnamed: 10',
    'unnamed:',
    'unnamed: 9',
    'macula_ids_in_protasis',
    'macula_ids_in_apodosis',
    'matched_english_in_protasis',
    'matched_english_in_apodosis',
    'matched_gloss_in_protasis',
    'matched_gloss_in_apodosis',
    'all_verse_word_tuples',
    'matches_map',
    'all_protasis_words',
    'all_apodosis_words',
    'unmatched_protasis_words',
    'unmatched_apodosis_words',
    'protasis',
    'apodosis',
]

columns_to_filter = [column for column in columns_to_filter if column in output_df_copy.columns]

output_df_copy.drop(columns=columns_to_filter, inplace=True)
# rename matched_protasis_words -> protasis	matched_apodosis_words -> apodosis
output_df_copy.rename(columns={'matched_protasis_words': 'protasis', 'matched_apodosis_words': 'apodosis'}, inplace=True)
# output_df_copy.head()

output_df_copy.to_html('output.html')
output_df_copy.to_csv('output.tsv', sep='\t', index=False)
